In [11]:
import pandas as pd
import psycopg2 as pg2

database = 'arch-snap_twitter-scale-1'
conn = pg2.connect("host='localhost'  port=5422 password='yuchao' dbname=%s"%database)

In [2]:
sql = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';"
pd.read_sql(sql, conn)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,circles,duke,None,False,False,False,False
1,public,edges,duke,None,False,False,False,False


In [3]:
sql = '''
SELECT
   TABLE_NAME, COLUMN_NAME
FROM
   information_schema.COLUMNS
   JOIN
       (SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema') AS tables
    ON tables.tablename = table_name
'''
pd.read_sql(sql, conn)

,table_name,column_name
0,circles,source
1,circles,circle
2,circles,node
3,edges,source
4,edges,fnode
5,edges,tnode


In [18]:
sql = "SELECT * FROM circles;"
pd.read_sql(sql, conn)

,source,circle,node
0,110755733,1,109768622
1,16193542,0,232923829
2,292560337,0,238345831
3,96483973,0,89936041
4,96483973,0,526018795
...,...,...,...
50848,15329577,1,37691773
50849,15329577,1,50392516
50850,15329577,1,34335068
50851,15329577,1,14585318


In [14]:
sql = "SELECT source FROM circles group by source;"
pd.read_sql(sql, conn)

,source
0,351092905
1,23334169
2,19563357
3,21094279
4,191924526
...,...
933,14371740
934,32574761
935,17403755
936,44130462


In [19]:
sql = "SELECT * FROM edges;"
pd.read_sql(sql, conn)

,source,fnode,tnode
0,110755733,35831704,16409683
1,110755733,35831704,19671129
2,110755733,3004231,16409683
3,110755733,24278900,149726145
4,110755733,24900321,28209292
...,...,...,...
2286904,15329577,17714569,455978461
2286905,15329577,247461127,126881128
2286906,15329577,30283686,112508240
2286907,15329577,17714569,13790282


In [94]:
sql = "SELECT count(*) FROM circles group by source, circle;"
pd.read_sql(sql, conn)

,count
0,21
1,22
2,1
3,3
4,1
...,...
188,4
189,1
190,4
191,4


In [20]:
sql = "SELECT * FROM edges;"
pd.read_sql(sql, conn)

,source,fnode,tnode
0,110755733,35831704,16409683
1,110755733,35831704,19671129
2,110755733,3004231,16409683
3,110755733,24278900,149726145
4,110755733,24900321,28209292
...,...,...,...
2286904,15329577,17714569,455978461
2286905,15329577,247461127,126881128
2286906,15329577,30283686,112508240
2286907,15329577,17714569,13790282


In [3]:
sql = "SELECT * FROM combined;"
pd.read_sql(sql, conn)

,fnode,tnode
0,214328887,34428380
1,17116707,28465635
2,380580781,18996905
3,221036078,153460275
4,107830991,17868918
...,...,...
2420761,99841247,154263215
2420762,99841247,194403468
2420763,99841247,180165101
2420764,99841247,253509115


In [23]:
# Count how many nodes appearing in circles
# Document: Nodes 4039
sql = '''
select count(distinct node) from circles;
'''
pd.read_sql(sql, conn)

,count
0,2884


In [26]:
# Count how many nodes appearing in edges
# Document: Nodes 4039
sql = '''
select count(distinct node) from (select fnode as node from edges UNION select tnode as node from edges) AS nodes;
'''
pd.read_sql(sql, conn)

,count
0,3959


In [40]:
# Count how many nodes appearing in combined
# Document: Nodes 4039
sql = '''
select count(distinct node) from (select fnode as node from combined UNION select tnode as node from combined) AS nodes;
'''
pd.read_sql(sql, conn)

,count
0,4039


In [36]:
# Count the unique edges
# Document: Edges 88234
sql = "SELECT COUNT(*) FROM (SELECT 1 FROM edges GROUP BY fnode, tnode) as uni;"
pd.read_sql(sql, conn)

,count
0,168486


In [12]:
# Count the union of edges and reversed edges
# Document: Edges 88234
sql = '''
WITH undirected_edges AS (SELECT * from edges UNION SELECT source, tnode as fnode, fnode as tnode from edges)
select count(*) from undirected_edges;
'''
pd.read_sql(sql, conn)

,count
0,170174


In [15]:
# Count the union of edges and reversed edges from combined
# Document: Edges 88234
sql = '''
WITH undirected_edges AS (SELECT * from combined UNION SELECT tnode as fnode, fnode as tnode from combined) 
select count(*) from undirected_edges;
'''
pd.read_sql(sql, conn)

,count
0,176468


In [16]:
# Count how many edges are the same in reverse
sql = '''
select count(*) from edges as e1 join edges as e2 on e1.fnode = e2.tnode AND e1.tnode = e2.fnode AND e1.source = e2.source;
'''
pd.read_sql(sql, conn)

,count
0,170174


In [17]:
# Count how many edges are the same in reverse without source restriction
sql = '''
select count(*) from edges as e1 join edges as e2 on e1.fnode = e2.tnode AND e1.tnode = e2.fnode;
'''
pd.read_sql(sql, conn)

,count
0,173634


In [5]:
# Count how many edges from combined are the same in reverse
sql = '''
select count(*) from combined as e1 join combined as e2 on e1.fnode = e2.tnode AND e1.tnode = e2.fnode;
'''
pd.read_sql(sql, conn)

,count
0,0


In [38]:
# Count the unique edges from combined
# Document: Edges 88234
sql = "SELECT COUNT(*) FROM (SELECT 1 FROM combined GROUP BY fnode, tnode) as uni;"
pd.read_sql(sql, conn)

,count
0,88234


In [21]:
# Count triangles from the same source
# Document: Number of triangles 1612010
sql = '''
select count(*) from edges as e1, edges as e2, edges as e3 WHERE e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode AND e1.source = e2.source AND e2.source = e3.source;
'''
pd.read_sql(sql, conn)

,count
0,9195840


In [22]:
# Count triangles
# Document: Number of triangles 1612010
sql = '''
select count(*) from edges as e1, edges as e2, edges as e3 WHERE e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode;
'''
pd.read_sql(sql, conn)

,count
0,9491958


In [16]:
# Count triangles
# Document: Number of triangles 1612010
sql = '''
WITH undirected_combined as (SELECT * from combined UNION select tnode as fnode, fnode as tnode from combined)
select count(*)/6 from undirected_combined as e1, undirected_combined as e2, undirected_combined as e3 WHERE e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode;
'''
pd.read_sql(sql, conn)

,?column?
0,1612010


In [7]:
# Test Union
sql = '''
select * from combined limit 1;
'''
pd.read_sql(sql, conn)

,fnode,tnode
0,0,1


In [11]:
# Test Union
sql = '''
(select tnode as fnode, fnode as tnode from combined limit 1)
UNION
(select * from combined limit 1);
'''
pd.read_sql(sql, conn)

,fnode,tnode
0,0,1
1,1,0


In [ ]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different circle
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''
WITH full_info AS (
    SELECT 
        edges.source, fnode, c1.circle as fcircle, tnode, c2.circle as tcircle 
    FROM
        edges, circles c1, circles c2 where edges.source = c1.source AND edges.source = c2.source AND edges.fnode = c1.node AND edges.tnode = c2.node
)
select 
    count(*) 
from 
    full_info e1, full_info e2, full_info e3
WHERE
    e1.fcircle = e1.tcircle
    AND e2.fcircle = e2.tcircle
    AND e3.fcircle = e3.tcircle
    AND e1.fcircle <> e2.fcircle
    AND e2.fcircle <> e3.fcircle
    AND e1.fcircle <> e3.fcircle
    AND e1.source = e2.source
    AND e2.source = e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
    ;
'''
pd.read_sql(sql, conn)

In [5]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different circle
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''
WITH full_info AS (
    SELECT 
        edges.source, fnode, c1.circle as fcircle, tnode, c2.circle as tcircle 
    FROM
        edges, circles c1, circles c2 
    where edges.source = c1.source AND edges.source = c2.source AND edges.fnode = c1.node AND edges.tnode = c2.node
        AND c1.source = c2.source       
)
select 
    count(*) 
from 
    full_info e1, full_info e2, full_info e3
WHERE
    True
    AND e1.fcircle < e2.fcircle
    AND e2.fcircle < e3.fcircle
    AND e1.source = e2.source
    AND e2.source = e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
    ;
'''
pd.read_sql(sql, conn)

,count
0,63679680


In [10]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different circle
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''
WITH full_info AS (
    SELECT 
        edges.source, fnode, c1.circle as fcircle, tnode, c2.circle as tcircle 
    FROM
        edges, circles c1, circles c2 
    where edges.source = c1.source AND edges.source = c2.source AND edges.fnode = c1.node AND edges.tnode = c2.node
        AND c1.circle = c2.circle       
)
select 
    e1.source as source, e1.fcircle as circle1, e2.fcircle as circle2, e3.fcircle as circle3, count(*) as cnt 
from 
    full_info e1, full_info e2, full_info e3
WHERE
    True
    AND e1.fcircle < e2.fcircle
    AND e2.fcircle < e3.fcircle
    AND e1.source = e2.source
    AND e2.source = e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
GROUP BY
    e1.source, e1.fcircle, e2.fcircle, e3.fcircle
;
'''
triangles = pd.read_sql(sql, conn)

In [11]:
triangles.groupby(['source'])['cnt'].agg('sum')

Series([], Name: cnt, dtype: float64)

In [30]:
triangles.sort_values(['cnt'], ascending=False)

,source,circle1,circle2,circle3,cnt
44,348,1,11,13,20751
20,348,1,2,11,10366
38,348,1,8,11,9374
4,348,0,1,11,8610
102,686,0,4,13,6235
...,...,...,...,...,...
298,1912,21,35,40,1
266,1912,10,35,43,1
12,348,0,7,13,1
9,348,0,2,13,1


In [31]:
with pd.option_context("display.max_rows", 1000):
    display(triangles.query('cnt > 1000').sort_values(['cnt'], ascending=False))

,source,circle1,circle2,circle3,cnt
44,348,1,11,13,20751
20,348,1,2,11,10366
38,348,1,8,11,9374
4,348,0,1,11,8610
102,686,0,4,13,6235
99,686,0,4,9,4744
101,686,0,4,12,3800
189,686,4,9,13,3719
40,348,1,8,13,2270
120,686,0,9,13,1844


In [ ]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different ego-network
# Output:
# source, circle 1, circle 2, circle 3, triangles
sql = '''SELECT e1.source as ego1, e2.source ego2, e3.source ego3, count(*) as triangles FROM edges e1, edges e2, edges e3
    WHERE e1.source < e2.source AND e2.source < e3.source
    AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
GROUP BY
    e1.source, e2.source, e3.source
;
'''
triangles_ego = pd.read_sql(sql, conn)

In [7]:
triangles_ego.sort_values(['triangles'], ascending=False)

,ego1,ego2,ego3,triangles


In [5]:
sql = 'select source from circles group by source limit 10;'
pd.read_sql(sql, conn)

,source
0,351092905
1,23334169
2,19563357
3,21094279
4,191924526
5,34416153
6,64441390
7,114885421
8,7670202
9,31610343


In [14]:
# Count how many triangles such that all individuals share the common friend, and each of edge from different ego-network
# Output:
# source, circle 1, circle 2, circle 3, triangles
import numpy as np

sql = 'select source from circles group by source limit 10;'
sources = pd.read_sql(sql, conn)['source'].tolist()

triangles_ego = None
for i in range(100):
    tri_sources = np.random.choice(sources, 3)
    a, b, c = tri_sources
    sql = '''SELECT e1.source as ego1, e2.source ego2, e3.source ego3, count(*) as triangles FROM edges e1, edges e2, edges e3
        WHERE e1.source = '%s' AND e2.source = '%s' AND e3.source = '%s'
        AND e1.tnode = e2.fnode AND e2.tnode = e3.fnode AND e3.tnode = e1.fnode
        GROUP BY
    e1.source, e2.source, e3.source
    ;
    '''%(a, b, c)
    if triangles_ego is not None:
        triangles_ego = triangles_ego.append(pd.read_sql(sql, conn))
    else:
        triangles_ego = pd.read_sql(sql, conn)

In [15]:
triangles_ego

,ego1,ego2,ego3,triangles
0,23334169,31610343,19563357,8
0,351092905,23334169,114885421,31
0,351092905,31610343,31610343,1110
0,351092905,19563357,351092905,208
0,19563357,31610343,114885421,3246
0,31610343,351092905,114885421,344
0,23334169,114885421,23334169,498
0,23334169,19563357,351092905,9
0,19563357,114885421,23334169,55
0,19563357,19563357,31610343,6763


In [27]:
triangles_ego.sort_values(['triangles'], ascending=False)

,ego1,ego2,ego3,triangles
